In [2]:
import pandas as pd
import numpy as np
import itertools
from sklearn import preprocessing

In [ ]:
%%time
#读取kegg数据
hsa = pd.read_csv('F:/MyData/deepSurvival/data/KEGG/KEGG_gene_hsa.csv', sep='\t', 
                  engine='python', header=None, index_col=None, names=['keggId', 'geneName'])
hsa_brite = pd.read_csv('F:/MyData/deepSurvival/data/KEGG/KEGG_hsa_brite.csv', 
                        sep='\t', engine='python', header=None, index_col=None, names=['keggBriteId', 'keggId'])
brite = pd.read_csv('F:/MyData/deepSurvival/data/KEGG/KEGG_brite.csv', 
                    engine='python', header=0, index_col=0)

In [ ]:
%%time
#读取基因表达数据
tcga_expre = pd.read_hdf("F:/MyData/deepSurvival/data/GSE37745/tcga_geo_expre.h5", 
                      key = "tcga")
geo_expre = pd.read_hdf("F:/MyData/deepSurvival/data/GSE37745/tcga_geo_expre.h5", 
                      key = "geo")

In [ ]:
tcga_expre

In [ ]:
geo_expre

In [ ]:
#替换负值
def replace_negative(df):
    # checking the element is < 0
    df[df < 0] = 0

In [ ]:
tcga_expre=tcga_expre.T

In [ ]:
tcga_expre

In [ ]:
geo_expre=geo_expre.T
geo_expre

In [ ]:
#from multiprocessing import Pool
#import time
#P=Pool(processes=8)
for i in list(tcga_expre.columns):
   # 获取各个指标的最大值和最小值
    Max = np.max(tcga_expre[i])
    Min = np.min(tcga_expre[i])
    tcga_expre[i] = (tcga_expre[i] - Min)/(Max - Min)

In [ ]:
tcga_expre=tcga_expre.fillna(0)

In [ ]:
tcga_expre

In [ ]:
for i in list(geo_expre.columns):
   # 获取各个指标的最大值和最小值
    Max = np.max(geo_expre[i])
    Min = np.min(geo_expre[i])
    geo_expre[i] = (geo_expre[i] - Min)/(Max - Min)

In [ ]:
geo_expre=geo_expre.fillna(0)

In [ ]:
geo_expre

In [ ]:
tcga_clinical=pd.read_hdf('E:/MyData/deepSurvival/data/Preprocessed_Data/Lung.h5',key='clinical')

In [ ]:
tcga_clinical

In [ ]:
len(tcga_clinical.index.intersection(list(map(lambda x: x.split('_')[0], tcga_expre.index))))

In [ ]:
ens_ids=tcga_expre.columns

In [ ]:
ens_ids

In [ ]:
print("The number of genes: {}".format(len(ens_ids)))

In [ ]:
print("Total number of samples (patients): {}".format(tcga_expre.shape[0]))

In [ ]:
%%time
#导入ensembol-hugo字典
hugo_to_ens = pd.read_csv('E:/MyData/deepSurvival/data/PanCancer/hugo_to_ens.tsv', 
                       sep='\t', engine='python', index_col=None)

In [ ]:
hugo_to_ens

In [ ]:
geneId=hugo_to_ens['geneId']

In [ ]:
ensembl=[]
for ids in geneId:
    ensembl.append(ids[0:15])

In [ ]:
ensembl

In [ ]:
hugo_to_ens['ensembl']=ensembl

In [ ]:
hugo_to_ens

In [ ]:
hugo_to_ens.drop_duplicates(subset='ensembl',keep='first')

In [ ]:
hugo_to_ens=hugo_to_ens.drop_duplicates(subset='ensembl',keep='first')

In [ ]:
hugo_to_ens = hugo_to_ens[hugo_to_ens['ensembl'].isin(ens_ids)]
hugo_to_ens.shape

In [ ]:
hugo_to_ens

In [ ]:
gene_mapping = hugo_to_ens[['ensembl', 'geneName']]
gene_mapping.head()

In [ ]:
sum(gene_mapping.duplicated())

In [ ]:
len(set(gene_mapping['geneName']))

In [ ]:
len(set(gene_mapping['ensembl']))

In [ ]:
sum(map(lambda x: ',' in x or ';' in x, gene_mapping.geneName))

In [ ]:
%%time
tcga_gene_map = tcga_expre.T
tcga_gene_map['ensembl'] = tcga_gene_map.index
tcga_gene_map['order'] = tcga_expre.mean(axis=0) # sort genes by mean expression values 
tcga_gene_map = pd.merge(gene_mapping, tcga_gene_map, on='ensembl')

In [ ]:
tcga_gene_map

In [ ]:
geo_clinical=pd.read_hdf('E:/MyData/deepSurvival/data/GSE37745/GSE37745.h5',key='clinical')

In [ ]:
geo_clinical

In [ ]:
len(geo_clinical.index.intersection(list(map(lambda x: x.split('_')[0], geo_expre.index))))

In [3]:
%%time
#导入ensembol-hugo字典
hugo_to_ens = pd.read_csv('F:/MyData/deepSurvival/data/PanCancer/hugo_to_ens.tsv', 
                       sep='\t', engine='python', index_col=None)

CPU times: total: 1.36 s
Wall time: 1.51 s


In [ ]:
geneId=hugo_to_ens['geneId']

In [ ]:
ensembl=[]
for ids in geneId:
    ensembl.append(ids[0:15])

In [ ]:
hugo_to_ens['ensembl']=ensembl

In [ ]:
hugo_to_ens

In [ ]:
hugo_to_ens=hugo_to_ens.drop_duplicates(subset='ensembl',keep='first')

In [ ]:
hugo_to_ens = hugo_to_ens[hugo_to_ens['ensembl'].isin(ens_ids)]
hugo_to_ens.shape

In [ ]:
gene_mapping = hugo_to_ens[['ensembl', 'geneName']]
gene_mapping.head()

In [ ]:
%%time
geo_gene_map = geo_expre.T
geo_gene_map['ensembl'] = geo_gene_map.index
geo_gene_map['order'] = geo_expre.mean(axis=0) # sort genes by mean expression values 
geo_gene_map = pd.merge(gene_mapping, geo_gene_map, on='ensembl')

In [ ]:
geo_gene_map

In [ ]:
%%time
# Save dataset
with pd.HDFStore('data/hugo_ensembl_geneExpre_map.h5','w') as store:
    store['tcga_gene_map']=tcga_gene_map
    #store['non_lung_gene_map_log2tpm']=non_lung_gene_map
    store['geo_gene_map']=geo_gene_map